In [ ]:
from flask import Flask, request, jsonify
import os
import json
from retrieval.local_retrieval import local_retrieve  # 本地检索
from retrieval.cloud_retrieval import cloud_retrieve  # 云端检索

app = Flask(__name__)

@app.route('/query', methods=['POST'])
def query():
    try:
        # 从前端获取查询和模型选择
        data = request.get_json()
        query = data.get("query")
        use_local = data.get("use_local", True)  # 用户选择是否使用本地模型

        # 根据选择调用本地模型或云端模型
        if use_local:
            # 使用本地检索
            results = local_retrieve(query)
        else:
            # 使用云端检索
            cloud_urls = json.loads(os.getenv('CLOUD_URLS', '[]'))  # 从环境变量读取云端 URL 列表
            results = cloud_retrieve(query, cloud_urls)
        
        # 返回结果
        answer = "\n".join([doc.page_content for doc in results]) if results else "⚠️ 无法找到相关文档。"

        return jsonify({"answer": answer})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)  # 启动 Flask 服务
